### Домашнее задание к вебинару 3

### 1) Попытаться ответить на вопросы/выдвинуть гипотезы по 5 товарам

**Ситуация**: Вы работает data scientist в крупном продуктовом российском ритейлере. Ваш конкурент сделал рекомендательную систему, и его продажи выросли. Ваш менеджмент тоже хочет увеличить продажи   
**Задача со слов менеджера**: Сделайте рекомендательную систему топ-10 товаров для рассылки по e-mail

**В итоге договорились с менеджером, что:**
- Хотим повысить выручку минимум на 6% за 4 месяца. Будем повышать за счет роста Retention минимум на  3% и среднего чека минимум на 3%
- Топ-5 товаров, а не топ-10 (В e-mail 10 выглядят не красиво, в push и на чек больше 5 не влезает)
- Рассылаем в e-mail (5% клиентов) и push-уведомлении (20% клиентов), печатаем на чеке (все оффлайн клиенты)

**Итак, необходимо включить в рассылку 5 товаров:** 
  
**3 товара** с акцией (Как это учесть? А если на товар была акция 10%, а потом 50%, что будет стоять в user-item матрице?)   
  
*Предполагаю, что нужно использовать акцию, актуальную на данный момент. Если на этой неделе скидка 50%, ее и учиываем при расчете.*

**1 новый товар** (юзер никогда не покупал. Просто фильтруем аутпут ALS? А если у таких товаров очень маленькая вероятность покупки? Может, использовать другую логику/модель?)  
  
*Я предполагаю, что необходимо отфильтровать по вероятности покупки и выбрать товар, который пользователь не покупал раньше, у которого будет наибольшая вероятность покупки (возможно, исходя из покупок похожих пользователей (соседей)). То есть использовать пост-фильтрацию*


**1 товар** для роста среднего чека (товары минимум дороже чем обычно покупает юзер. Как это измерить? На сколько дороже?)  
  
*Эта дельта в стоимости должна быть адекватной для юзера, то есть необходимо максимально корректно и этично ее рассчитать, чтобы цена была адекватна сегменту покупателя. Предложив товар дороже более чем в 2 раза от обычной аналогичной покупки, мы рискуем оттолкнуть клиента.*  
*Предполагаю, что лучше всего выбирать такой товар не радомно, а из категории, в котором у юзера есть покупки. То есть, если он часто покупает сыр, предложить ему сыр чуть выше стоимостью.*  

*Дельту можно рассчитать исходя из среднего чека. Допустим, если клиент покупает сок по 100 руб, однако его средний чек 4000 руб, вполне допустимо предложить ему сок дороже, нежели клиенту, покупающему такой же сок, но со средним чеком 2000 руб. То есть возможно, второй клиент любит сок, но в целом тратит меньше денег в магазине*

### 4) Поэкспериментировать с ALS (grid-search)

In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

from sklearn.model_selection import GridSearchCV, KFold

# Функции из 1-ого вебинара
import os, sys

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from src.metrics import precision_at_k, recall_at_k

In [21]:
DATASET_PATH = './retail_train.csv'
ITEMS_PATH = './product.csv'

In [86]:
data = pd.read_csv(DATASET_PATH)
items_data = pd.read_csv(ITEMS_PATH)

In [23]:
items_data.columns = [col.lower() for col in items_data.columns]
items_data.rename(columns = {'product_id':'item_id'}, inplace=True)
items_data.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [24]:
data.columns = [col.lower() for col in data.columns]
data.rename(columns={'household_key': 'user_id',
                    'product_id': 'item_id'},
           inplace=True)


test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [25]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [26]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

In [27]:
# Заведем фиктивный item_id (если юзер покупал товары из топ-5000, то он "купил" такой товар)
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999



/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


**Создадим функцию для префильтрации данных**

In [28]:
# Создаем user_item matrix 
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # необязательно использовать именно quantity
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат sparse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(3)

item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [29]:
#Создаем словари для перевода исходные индексы в матричные

userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

matrixid_to_itemid = dict(zip(matrix_itemids, itemids))
matrixid_to_userid = dict(zip(matrix_userids, userids))

itemid_to_matrixid = dict(zip(itemids, matrix_itemids))
userid_to_matrixid = dict(zip(userids, matrix_userids))

In [30]:
model = AlternatingLeastSquares(factors=100, 
                                regularization=0.001,
                                iterations=15, 
                                calculate_training_loss=True, 
                                num_threads=4)

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

  0%|          | 0/15 [00:00<?, ?it/s]

In [31]:

#Создадим функцию для получения рекомендаций для всех пользователей
def get_recommendations(user, model, N=5):
    res = [matrixid_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_matrixid[user], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=N, 
                                    filter_already_liked_items=False, 
                                    filter_items=[itemid_to_matrixid[999999]], 
                                    recalculate_user=True)]
    return res

In [32]:
result['als'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))

In [33]:
result.apply(lambda row: precision_at_k(row['als'], row['actual']), axis=1).mean()

0.1791380999020568

#### Попробуем подобрать лучшие гиперпараметры для модели

In [210]:
params = {'factors':[35, 55],
         'regularization':[0.01, 0.05, 0.1],
         'iterations':[10, 15, 30]}

# перебрала большое количество параметров, оставила лучшие

In [211]:
%%time

for factor in params['factors']:
    for reg in params['regularization']:
        for itr in params['iterations']:
            model = AlternatingLeastSquares(factors=factor, 
                                            regularization=reg,
                                            iterations=itr, 
                                            calculate_training_loss=True, 
                                            num_threads=4)

            model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
                      show_progress=False)
            
            result['als'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))
            metric_value = result.apply(lambda row: precision_at_k(row['als'], row['actual']), axis=1).mean()
            print(f'factors={factor}, regularization={reg}, iterations={itr}: {metric_value:.6f}.')

factors=35, regularization=0.01, iterations=10: 0.190597.
factors=35, regularization=0.01, iterations=15: 0.184427.
factors=35, regularization=0.01, iterations=30: 0.185798.
factors=35, regularization=0.05, iterations=10: 0.188345.
factors=35, regularization=0.05, iterations=15: 0.187071.
factors=35, regularization=0.05, iterations=30: 0.187365.
factors=35, regularization=0.1, iterations=10: 0.190891.
factors=35, regularization=0.1, iterations=15: 0.188541.
factors=35, regularization=0.1, iterations=30: 0.188541.
factors=55, regularization=0.01, iterations=10: 0.190010.
factors=55, regularization=0.01, iterations=15: 0.184721.
factors=55, regularization=0.01, iterations=30: 0.182860.
factors=55, regularization=0.05, iterations=10: 0.188834.
factors=55, regularization=0.05, iterations=15: 0.188443.
factors=55, regularization=0.05, iterations=30: 0.185896.
factors=55, regularization=0.1, iterations=10: 0.188737.
factors=55, regularization=0.1, iterations=15: 0.184721.
factors=55, regular

**Вывод:** В целом, значения метрики достаточно неплохие.  
Наибольший precision_at_k = 0.190891 достигается при параметрах:  
factors = 35,  
regularization = 0.1,  
iterations = 10

In [212]:
result.head(2)

,user_id,actual,als
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[965766, 1033142, 5569374, 1005186, 832678]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[951590, 1106523, 1133018, 1092026, 5568378]"


In [36]:
precision_scores = pd.DataFrame({'model' : ['als'],
                         'precision@k_score': 0.190891})
precision_scores

,model,precision@k_score
0,als,0.190891


#### Попробуем сделать взвешивание user_item_matrix

In [34]:
#Обновим нашу модель ALS

model = AlternatingLeastSquares(factors = 35,
                              regularization = 0.1,
                              iterations = 10,
                              calculate_training_loss = True,
                              num_threads = 4,
                              )

#### Сделаем TFIDF взвешивание user_item_matrix

In [47]:
user_item_matrix_tfidfweighted = tfidf_weight(user_item_matrix.T).T
sparse_user_item = csr_matrix(user_item_matrix_tfidfweighted).tocsr()

In [48]:
# Обучим модель на новой взвешенной матрице
model.fit(csr_matrix(user_item_matrix_tfidfweighted).T.tocsr(), 
         show_progress = True)

  0%|          | 0/10 [00:00<?, ?it/s]

In [49]:
result['als_with_tfidf'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))
#result.apply(lambda row: precision_at_k(row['als_with_tfidf'], row['actual']), axis=1).mean()

In [50]:
precision_k_tfidf = result.apply(lambda row: precision_at_k(row['als_with_tfidf'], row['actual']), axis=1).mean()

In [51]:
precision_scores.loc[precision_scores.shape[0]] = ['als_with_tfidf', precision_k_tfidf]

#### Теперь сделаем BM25 взвешивание

In [52]:
user_item_matrix_bm25weighted = bm25_weight(user_item_matrix.T).T
sparse_user_item = csr_matrix(user_item_matrix_bm25weighted).tocsr()

In [53]:
model.fit(csr_matrix(user_item_matrix_bm25weighted).T.tocsr(), 
         show_progress = True)

  0%|          | 0/10 [00:00<?, ?it/s]

In [54]:
result['als_with_bm25'] = result['user_id'].apply(lambda x: get_recommendations(x, model = model, N=5))
#result.apply(lambda row: precision_at_k(row['als_with_bm25'], row['actual']), axis = 1).mean()

In [55]:
precision_k_bm25 = result.apply(lambda row: precision_at_k(row['als_with_bm25'], row['actual']), axis = 1).mean()

In [56]:
precision_scores.loc[precision_scores.shape[0]] = ['als_with_bm25', precision_k_bm25]

In [61]:
precision_scores

,model,precision@k_score
2,als_with_bm25,0.198041
3,als_with_tfidf,0.173164
4,als_with_bm25,0.128991


**Вывод:** делаем вывод, что взвешивание понизило результат

### Сделаем рекомендации с предварительной фильтрацией датасета

In [76]:
#Переопределим датафрейм
data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]

In [77]:
def prefilter_items(data):
    
    fake_id = 99999
    
    # 1. Уберем самые популярные товары (их и так купят)
    #popularity = data.groupby('item_id')['user_id'].nunique().reset_index() / data['user_id'].nunique()
    #popularity.rename(columns={'user_id': 'unique_buyers'}, inplace=True)
    
    #top_popular = popularity[popularity['unique_buyers'] > 0.4].item_id.tolist()
    
    #data = data[~data['item_id'].isin(top_popular)]
    #data.loc[data['item_id'].isin(top_popular)] = fake_id
    
    # 2. Уберем самые НЕ популярные товары (их и так НЕ купят)
    #top_notpopular = popularity[popularity['unique_buyers'] < 0.02].item_id.tolist()
    
    #data = data[~data['item_id'].isin(top_notpopular)]
    #data.loc[data['item_id'].isin(top_popular)] = fake_id
    
    # 3. Уберем товары, которые не продавались за последние 12 месяцев
    ''' 12 месяцев = 52 недели '''
    
    max_week = data['week_no'].max()
    max_week_for_item = data_train.groupby('item_id')['week_no'].max().reset_index()
    
    '''Выделим товары, которые продавались последний раз менее года назад'''
    new_items = max_week_for_item.loc[max_week_for_item['week_no'] > max_week - 52, 'item_id'].tolist()
    
    #data.loc[~data['item_id'].isin(new_items), 'item_id'] = fake_id
    
    data = data[data['item_id'].isin(new_items)]
    
    # 4. Уберем не интересные для рекоммендаций категории (department)
    department_size = items_data.groupby('department')['item_id'].nunique().reset_index()
    department_size.columns = ['department', 'n_items']
    rare_departments = department_size[department_size['n_items'] < 150].department.tolist()
    items_in_rare_departments = items_data[items_data['department'].isin(rare_departments)].item_id.unique().tolist()
    data = data[~data['item_id'].isin(items_in_rare_departments)]
    
    
    # 5. Уберем слишком дешевые и слишком дорогие товары (на них не заработаем). 1 покупка из рассылок стоит 60 руб. 
    
    '''Найдем среднюю цену за товар, т.к. клиенты могли покупать по разным ценам'''
                  
    sales = data.groupby('item_id')[['sales_value', 'quantity']].sum().reset_index()
    sales.rename(columns={'sales_value': 'total_revenue', 'quantity': 'n_sold'}, inplace=True)
    sales['avg_price'] = np.where(sales['n_sold'] > 0, sales['total_revenue'] / sales['n_sold'], 0)
    ids_for_rec = sales[(sales['avg_price'] > 2) & (sales['avg_price'] < 50)].item_id.tolist()
    
    data = data[data['item_id'].isin(ids_for_rec)]
    #data.loc[~data['item_id'].isin(ids_for_rec), 'item_id'] = fake_id   

    # 6. Отфильтруем топ 5000
    popularity = data.groupby('item_id')['quantity'].sum().reset_index()
    popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)
    top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()
    
    # заведем фиктивный товар - все товары, которые не попали в топ 5000, назовем 999999
    data.loc[~data['item_id'].isin(top_5000), 'item_id'] = fake_id              
                    
    return data
    
def postfilter_items(user_id, recommendations):
    pass

In [78]:
# Сделаем префильтрацию 

data_train_ = prefilter_items(data_train)

In [79]:
new_test_users = set(data_test['user_id']) - set(data_train_['user_id'])
new_test_users

{1984, 2259}

In [80]:
#Уберем из result юзеров, которых мы "не знаем" (не встречали в тренировочном датасете)

result = result.loc[result['user_id'] != 1984]
result = result.loc[result['user_id'] != 2259]

In [81]:
# Создаем user_item matrix 
user_item_matrix = pd.pivot_table(data_train_, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # необязательно использовать именно quantity
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(3)

item_id,99999,117847,818981,818996,819255,819308,819400,819487,819590,819845,...,15926863,15926886,15926887,15926927,15972074,15972298,15972565,15972790,16100266,16729415
user_id,,,,,,,,,,,,,,,,,,,,,
1,231.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
2,125.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,30.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [82]:
#Создаем словари для перевода исходные индексы в матричные

userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

matrixid_to_itemid = dict(zip(matrix_itemids, itemids))
matrixid_to_userid = dict(zip(matrix_userids, userids))

itemid_to_matrixid = dict(zip(itemids, matrix_itemids))
userid_to_matrixid = dict(zip(userids, matrix_userids))

In [83]:
model = AlternatingLeastSquares(factors=35, 
                                regularization=0.1,
                                iterations=10, 
                                calculate_training_loss=True, 
                                num_threads=4)

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

  0%|          | 0/10 [00:00<?, ?it/s]

In [84]:
#Создадим функцию для получения рекомендаций для всех пользователей

def get_recommendations(user, model, N=5):
    res = [matrixid_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_matrixid[user], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=N, 
                                    filter_already_liked_items=False, 
                                    filter_items=[itemid_to_matrixid[99999]], 
                                    recalculate_user=True)]
    return res

In [85]:
result['als_after_filter'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))
result.apply(lambda row: precision_at_k(row['als_after_filter'], row['actual']), axis=1).mean()

0.13999999999999999

**Вывод:** префильтрация понизила результат